<a href="https://colab.research.google.com/github/ITCUI-XJTLU/A-Journey-to-Help-Finding-the-Members-of-the-Praesepe-Star-Cluster/blob/main/DAESC_Cui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
! pip install cupy-cuda12x

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
path="/content/drive/My Drive/Stat_Gene/Data"

os.chdir(path)
os.listdir(path)

['geneAE_allchr_bytopexonic.rda',
 'cellgene_info.rda',
 'ynx.csv',
 'id1hot.csv',
 'cell_metadata_cols.tsv',
 'ynxid_data5.csv',
 'ynxid_data5.npy',
 'id_data5.npy',
 'ynxid_allgenes.npy',
 'id_alldata.npy',
 'ynxid_data10.csv',
 'ynxid_genes10.npy',
 'id_data10.npy']

In [32]:
import time
import numpy as np
from csv import reader
from scipy import special
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [4]:
import cupy as cp

from cupyx.scipy.special import _digamma
from cupyx.scipy.special import _gamma
from cupyx.scipy.special import _zeta

In [40]:
import pdb

# One Gene + Cupy

In [5]:
# Import Data
# get x, y, n
ynxid_data = np.load("ynxid_genes10.npy")
print(ynxid_data.shape)

# get id
id_data = np.load("id_data10.npy")
cu_id_data = cp.asarray(id_data)
print(id_data.shape)
# id_data = id_data.T
#cu_id_data = cp.asarray(id_data)

(22, 30470)
(30470, 104)


In [6]:
# Import Data
gene_index = 6
#
genes_num = int((ynxid_data.shape[0]-2)/2)

cu_b_phi = cp.array([0.08, 0.5, 0.35]).astype(cp.float32)
cu_sigm2 = cp.array([0.5], dtype = cp.float32)


cu_x_0 = cp.zeros((ynxid_data.shape[1])) + 1
cu_x_1 = cp.asarray(ynxid_data[-2,:])
cu_x = cp.hstack((cu_x_0.reshape(-1, 1), cu_x_1.reshape(-1, 1))).astype(cp.float32)
cu_y = cp.asarray(ynxid_data[gene_index,:]).astype(cp.float32)
cu_n = cp.asarray(ynxid_data[gene_index+genes_num, :]).astype(cp.float32)


cu_randint = cp.zeros(cu_id_data.shape[1],dtype=cp.float32)

cu_ghq_weights = cp.array([0.1666667, 0.6666667, 0.1666667]).astype(cp.float32)
cu_ghq_nodes = cp.array([-1.732051, -2.045201e-16, 1.732051]).astype(cp.float32)

'''
with open("ynx.csv",'rt',encoding = 'UTF-8' )as raw_data:
  readers=reader(raw_data,delimiter=',')
  x=list(readers)
  ynx_data = np.array(x)
  ynx_data = ynx_data.astype(np.float32)
with open("id1hot.csv",'rt',encoding = 'UTF-8' )as raw_data:
  readers=reader(raw_data,delimiter=',')
  x=list(readers)
  id_data = np.array(x)
  id_data = id_data.astype(np.float32)

## Prepare for Cupy
cu_ynx_data = cp.asarray(ynx_data)
cu_id_data = cp.asarray(id_data).astype(cp.float32)

cu_b_phi = cp.array([0.08, 0.5, 0.35]).astype(cp.float32)
cu_sigm2 = cp.array([0.5], dtype = cp.float32)


cu_x_0 = cp.zeros((cu_ynx_data.shape[0])) + 1
cu_x_1 = cu_ynx_data[:,2]
cu_x = cp.hstack((cu_x_0.reshape(-1, 1), cu_x_1.reshape(-1, 1))).astype(cp.float32)
cu_y = cu_ynx_data[:,0].astype(cp.float32)
cu_n = cu_ynx_data[:,1].astype(cp.float32)


cu_randint = cp.zeros(cu_id_data.shape[1],dtype=cp.float32)

cu_ghq_weights = cp.array([0.1666667, 0.6666667, 0.1666667]).astype(cp.float32)
cu_ghq_nodes = cp.array([-1.732051, -2.045201e-16, 1.732051]).astype(cp.float32)
'''



'\nwith open("ynx.csv",\'rt\',encoding = \'UTF-8\' )as raw_data:\n  readers=reader(raw_data,delimiter=\',\')\n  x=list(readers)\n  ynx_data = np.array(x)\n  ynx_data = ynx_data.astype(np.float32)\nwith open("id1hot.csv",\'rt\',encoding = \'UTF-8\' )as raw_data:\n  readers=reader(raw_data,delimiter=\',\')\n  x=list(readers)\n  id_data = np.array(x)\n  id_data = id_data.astype(np.float32)\n\n## Prepare for Cupy\ncu_ynx_data = cp.asarray(ynx_data)\ncu_id_data = cp.asarray(id_data).astype(cp.float32)\n\ncu_b_phi = cp.array([0.08, 0.5, 0.35]).astype(cp.float32)\ncu_sigm2 = cp.array([0.5], dtype = cp.float32)\n\n\ncu_x_0 = cp.zeros((cu_ynx_data.shape[0])) + 1\ncu_x_1 = cu_ynx_data[:,2]\ncu_x = cp.hstack((cu_x_0.reshape(-1, 1), cu_x_1.reshape(-1, 1))).astype(cp.float32)\ncu_y = cu_ynx_data[:,0].astype(cp.float32)\ncu_n = cu_ynx_data[:,1].astype(cp.float32)\n\n\ncu_randint = cp.zeros(cu_id_data.shape[1],dtype=cp.float32)\n\ncu_ghq_weights = cp.array([0.1666667, 0.6666667, 0.1666667]).astype(cp

In [10]:
def cu_trigamma(cu_array): # Test numpy(): trigamma(x_cpu) + x_gpu
  n = 1
  polygamma_cui = (-1.0)**(n+1) * _gamma.gamma(n+1.0) * _zeta.zeta(n+1.0, cu_array)
  return polygamma_cui.astype(cp.float32)

def cu_compute_randint_deriv_numpy(cu_e_b, cu_e_x, cu_e_randint, cu_id_data, cu_e_y, cu_e_n, cu_e_phi):
  cu_e_mu = cp.reciprocal( 1 +  cp.exp( - cp.dot(cu_e_x, cp.asarray(cu_e_b)).astype(cp.float32)
                                        - cp.dot(cu_id_data, cu_e_randint)))
  cu_e_mu = cp.clip(cu_e_mu,0.0000001, 0.9999999)

  cu_temp_1 = (special.digamma(cu_e_mu / cu_e_phi + cu_e_y) -
               special.digamma(( 1 - cu_e_mu)/ cu_e_phi + cu_e_n - cu_e_y) -
               special.digamma( cu_e_mu / cu_e_phi) +
               special.digamma((1 - cu_e_mu) / cu_e_phi) )
  cu_temp_2 = ( cu_trigamma(cu_e_mu / cu_e_phi + cu_e_y ) +
                cu_trigamma( ( 1 - cu_e_mu ) / cu_e_phi + cu_e_n - cu_e_y) -
                cu_trigamma( cu_e_mu / cu_e_phi) -
                cu_trigamma( ( 1 - cu_e_mu)/ cu_e_phi))

  cu_v1 = cp.multiply(cp.multiply(cu_temp_1, cu_e_mu), 1 - cu_e_mu) / cu_e_phi
  cu_v2 = (cp.multiply(cp.multiply(cu_temp_2, cp.multiply(cu_e_mu, cu_e_mu)), cp.multiply(1- cu_e_mu, 1-cu_e_mu)) / (cu_e_phi * cu_e_phi) +
           cp.multiply(cp.multiply(cp.multiply(cu_temp_1, 1 - 2 * cu_e_mu), cu_e_mu), 1 - cu_e_mu) / cu_e_phi )
  cu_v1 = cp.sum( cp.multiply(cu_id_data.transpose(), cu_v1), axis = 1)
  cu_v2 = cp.sum( cp.multiply(cu_id_data.transpose(), cu_v2), axis = 1)

  cu_v = cp.hstack((cu_v1.reshape(-1, 1), cu_v2.reshape(-1, 1)))
  return cu_v

def cu_q_function(param,cu_x_q,cu_y_q,cu_n_q,cu_zmat_q,cu_ghq_weights_q, cu_randint_q,cu_randint_prec_q,cu_id_data_q):
    print("q function param: " +str(param))
    cu_b_q = cp.asarray(param[0:2]).astype(cp.float32)
    cu_phi_q = cp.abs(cp.asarray(param[-1])).astype(cp.float32)
    # print("step 0 time : {} ms".format(1000*(end_time0 - start_time1)))

    cu_linpred_q = cp.dot(cu_x_q,cu_b_q)
    # print("step 1 time : {} ms".format(1000*(end_time1 - start_time1)))

    cu_mu_q = cp.reciprocal(1 + cp.exp(- ( cu_linpred_q.reshape(-1, 1) +
                                  cp.dot(cu_id_data_q,cu_randint_q).reshape(-1, 1) +
                                  cp.divide(cu_zmat_q, cp.dot(cu_id_data_q, cp.sqrt(cu_randint_prec_q)).reshape(-1, 1)))))
    # print("step 2 time : {} ms".format(1000*(end_time2 - start_time1)))

    cu_mu_q = cp.clip(cu_mu_q,1e-6,1-1e-6)
    # print("step 3 time : {} ms".format(1000*(end_time3 - start_time1)))

    cu_alpha_1_q = cu_mu_q / cu_phi_q
    cu_alpha_2_q = (1 - cu_mu_q) / cu_phi_q
    # print("step 4 time : {} ms".format(1000*(end_time4 - start_time1)))

    cu_q_func_result = cp.dot(cp.sum(special.betaln(cu_alpha_1_q+cu_y_q,cu_alpha_2_q + cu_n_q - cu_y_q)
                                  -special.betaln(cu_alpha_1_q,cu_alpha_2_q),axis=0),
                              cu_ghq_weights_q)
    # print("step 5 time : {} ms".format(1000*(end_time5 - start_time1)))
    print("q function : \n" + str(- cu_q_func_result))
    return cp.asnumpy(- cu_q_func_result).astype(np.float32)

def cu_q_function_deri(param,cu_x_q,cu_y_q,cu_n_q,cu_zmat_q,cu_ghq_weights_q,cu_randint_q,cu_randint_prec_q,cu_id_data_q):
    start_time1 = time.time()
    print("q deriv param: \n" + str(param))
    cu_b_q = cp.asarray(param[0:2]).astype(cp.float32)
    cu_phi_q = cp.asarray(param[-1]).astype(cp.float32)
    cu_linpred_q = cp.dot(cu_x_q,cu_b_q)
    #print("deri cu_linpred_q dtype: " +str(cu_linpred_q.dtype))

    cu_mu_q = cp.reciprocal(1 + cp.exp(- ( cu_linpred_q.reshape(-1, 1) +
                                  cp.dot(cu_id_data_q,cu_randint_q).reshape(-1, 1) +
                                  cp.divide(cu_zmat_q, cp.dot(cu_id_data_q, cp.sqrt(cu_randint_prec_q)).reshape(-1, 1)))))

    cu_mu_q = cp.clip(cu_mu_q,1e-6,1-1e-6)

    cu_alpha_1_q = cu_mu_q / cu_phi_q
    cu_alpha_2_q = (1 - cu_mu_q) / cu_phi_q
    # print("cu_alpha2_q dtype: " + str(cu_alpha_1_q.dtype))

    cu_temp_b_q = special.digamma(cu_alpha_1_q+cu_y_q) - special.digamma(cu_alpha_2_q+cu_n_q-cu_y_q) - special.digamma(cu_alpha_1_q) + special.digamma(cu_alpha_2_q)
    cu_deri_b_q = cp.dot((cp.dot(cu_x_q.transpose(),cp.multiply(cp.multiply(cu_temp_b_q,cu_mu_q),1-cu_mu_q))/cu_phi_q),cu_ghq_weights_q)

    cu_temp_phi_q = cp.sum((special.digamma(cu_alpha_1_q)-special.digamma(cu_alpha_1_q+cu_y_q))*cu_mu_q +
              (special.digamma(cu_alpha_2_q)-special.digamma(cu_alpha_2_q+cu_n_q-cu_y_q))*(1 - cu_mu_q)+
              special.digamma(1/cu_phi_q+cu_n_q.reshape(-1,1))-special.digamma(1/cu_phi_q + cp.zeros((cu_mu_q.shape[0],cu_mu_q.shape[1]))),axis=0,dtype=cp.float32) / (cu_phi_q * cu_phi_q)

    # pdb.set_trace()
    cu_deri_phi_q = cp.dot(cu_temp_phi_q.transpose(),cu_ghq_weights_q)
    # print("cu_deri_phi_q datatype: " +str(cu_deri_phi_q.dtype))

    cu_deri_b_phi_q = cp.append(cu_deri_b_q,cu_deri_phi_q)
    end_time = time.time()
    print("derivative : \n" + str(- cu_deri_b_phi_q))
    print()
    return cp.asnumpy(- cu_deri_b_phi_q).astype(np.float32)

def cu_vem(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_ghq_weights, cu_ghq_nodes,n_iter,n_lap):
  # E-M Algorithm
  print("b: " + str(cu_b_phi[:2]) +  "  phi: " + str(cu_b_phi[2]) +  "  sigma2: " + str(cu_sigm2) )
  for inter in range(n_iter):
    start_time = time.time()

    # E-step
    start_e = time.time()
    cu_b = cu_b_phi[:2]
    cu_phi = cu_b_phi[2]
    for i in range(n_lap):
      cu_df = cu_compute_randint_deriv_numpy(cu_b, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
      cu_randint = cu_randint - 0.9 * (cu_df[:,0] - cu_randint / cu_sigm2) / (cu_df[:,1] - 1 / cu_sigm2)
    cu_df = cu_compute_randint_deriv_numpy(cu_b, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
    cu_randint_prec = cp.abs(cu_df[:,1] - 1 / cu_sigm2)
    end_e = time.time()
    print("E-step : {} ms".format(1000*(end_e - start_e)) )

    # M - step
    start_m = time.time()
    cu_zmat = cp.multiply(cp.zeros((cu_y.shape[0],3)) + 1, cu_ghq_nodes).astype(cp.float32)
    cu_sigm2 =  cp.mean( cu_randint * cu_randint + cp.reciprocal(cu_randint_prec) )
    b_phi = cp.asnumpy(cu_b_phi).astype(np.float32)
    cu_optim_result = minimize(cu_q_function, b_phi, method='L-BFGS-B',jac = cu_q_function_deri,
                       args = (cu_x, cu_y.reshape(-1,1), cu_n.reshape(-1,1),cu_zmat,
                               cu_ghq_weights, cu_randint, cu_randint_prec, cu_id_data))
    end_m= time.time()
    print("M-step : {} ms".format(1000*(end_m - start_m)))
    cu_b_phi = cu_optim_result.x
    cu_func =  cu_optim_result.fun
    end_time = time.time()
    print("b: " + str(cu_b_phi[:2]) +
          "  phi: " + str(cu_b_phi[2]) +
          "  sigma2: " + str(cu_sigm2) +
          "  Likelihood: " + str(cu_func)+
          "Time :{} ms".format(1000*(end_time - start_time)))
    print(cu_optim_result)
    print()
    return cu_optim_result



In [11]:
#
my_optim_result =  cu_vem(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n,
                          cu_ghq_weights, cu_ghq_nodes,n_iter = 1 ,n_lap = 2)

b: [0.08 0.5 ]  phi: 0.35  sigma2: [0.5]
E-step : 8.982658386230469 ms
q function param: [0.08       0.5        0.34999999]
q function : 
340615.08547854924
q deriv param: 
[0.08       0.5        0.34999999]
derivative : 
[  53.60634793  402.81424528 5900.28613281]

q function param: [ 0.07093609  0.43189105 -0.64763672]
q function : 
342523.32800457353
q deriv param: 
[ 0.07093609  0.43189105 -0.64763672]
derivative : 
[-8537.42952793 -1725.52547632 86708.578125  ]

q function param: [0.07973085 0.49797752 0.32037536]
q function : 
340445.49619103415
q deriv param: 
[0.07973085 0.49797752 0.32037536]
derivative : 
[  55.14964273  421.98377558 5462.48632812]

q function param: [0.07826246 0.48694365 0.15875499]
q function : 
340131.02337260405
q deriv param: 
[0.07826246 0.48694365 0.15875499]
derivative : 
[   60.04948878   634.64689398 -5775.90917969]

q function param: [0.07901237 0.49257867 0.24129487]
q function : 
340093.5046222461
q deriv param: 
[0.07901237 0.49257867 0.2412948

In [ ]:
ynxid_data[0,:10]

array([0., 4., 4., 3., 2., 2., 3., 0., 3., 0.])

# Five Genes + numpy

In [ ]:
# Import Data
ynxid_data = np.load("ynxid_data5.npy")

id_data = np.load("id_data5.npy")
id_data = id_data.T
# Now, let's start our Game !

In [ ]:
ynxid_data

In [ ]:
b = np.array([[0.03, 0.4],[0.03, 0.4],[0.03, 0.4],[0.03, 0.4],[0.03, 0.4]], dtype=np.float32)
b = b.reshape(5,1,2)

phi = np.array([[0.72],[0.72],[0.72],[0.72],[0.72]],dtype = np.float32)
param = np.array([[0.03, 0.4, 0.72],[0.03, 0.4, 0.72],
                  [0.03, 0.4, 0.72],[0.03, 0.4, 0.72],
                  [0.03, 0.4, 0.72]]).astype(np.float32) # b0, b1, phi
sigm2 = np.array([[0.14],[0.14],[0.14],[0.14],[0.14]], dtype = np.float32)

x = np.zeros((5,ynxid_data.shape[1],2)).astype(np.float32)
for i in range(5):
  x_0 = np.zeros((ynxid_data.shape[1])) + 1
  x_1 = ynxid_data[10,:]
  x[i,:,:] = np.hstack((x_0.reshape(-1, 1), x_1.reshape(-1, 1))).astype(np.float32)

y = ynxid_data[0:5,:]
n = ynxid_data[5:10,:]

randint = np.zeros((5,id_data.shape[0]),dtype=np.float32)

ghq_weights = np.array([[0.1666667, 0.6666667, 0.1666667]]).astype(np.float32).reshape(1,3)

ghq_nodes = np.array([[-1.732051, -2.045201e-16, 1.732051]]).astype(np.float32).reshape(1,3)

In [ ]:
def estep(e_b, e_x, e_randint, id_data, e_y, e_n, e_phi, e_sigm2):
  for i in range(2):
    df = compute_randint_deriv_numpy(e_b, e_x, e_randint, id_data, e_y, e_n, e_phi)
    e_randint = e_randint - 0.9 * (df[:5,:] - e_randint / e_sigm2) / (df[5:,:] - 1 / e_sigm2)
  df = compute_randint_deriv_numpy(e_b, e_x, e_randint, cu_id_data, e_y, e_n, e_phi)
  e_randint_prec = np.abs(df[5:,:] - 1 / e_sigm2)
  return np.concatenate((e_randint , e_randint_prec),axis = 0)


def compute_randint_deriv_numpy(e_b, e_x, e_randint, id_data, e_y, e_n, e_phi):
  e_mu = np.reciprocal( 1 + np.exp(- np.sum(np.multiply(e_x,e_b),axis=2) - np.matmul(e_randint, id_data)))

  e_mu = np.clip(e_mu,0.0000001, 0.9999999)

  temp_1 = (special.digamma(e_mu / e_phi + e_y) -
          special.digamma((1-e_mu)/e_phi + e_n - e_y) -
          special.digamma(e_mu / e_phi) +
          special.digamma((1 - e_mu) / e_phi) )
  temp_2 = ( special.polygamma(1, e_mu / e_phi + e_y ) +
             special.polygamma(1, ( 1 - e_mu ) / e_phi + e_n - e_y) -
             special.polygamma(1, e_mu / e_phi) -
             special.polygamma(1, ( 1 - e_mu)/ e_phi) )

  v1 = temp_1 * e_mu * (1 - e_mu) / e_phi
  v2 = (np.multiply(np.multiply(temp_2, np.multiply(e_mu,e_mu)), np.multiply(1-e_mu, 1-e_mu)) / (e_phi * e_phi) +
        np.multiply(np.multiply(np.multiply(temp_1, 1 - 2 * e_mu), e_mu), 1 - e_mu) / e_phi )
  v1 = np.matmul(v1,id_data.T)
  v2 = np.matmul(v2,id_data.T)
  v = np.concatenate((v1,v2),axis=0)
  return v

# Q-function for numpy
def q_function(param, x_q, y_q, n_q, zmat_q, ghq_weights_q,randint_q, randint_prec_q, id_data_q):
    start_time1 = time.time()
    # print("param : "+ str(param))
    #数据升维
    param = param.reshape(5,-1) # 我们输入的参数必须为一维的，这里我们把参数升级为原本的二维
    b_q = param[:,:2]
    b_q = b_q.reshape(b_q.shape[0],1,b_q.shape[1]) # b shape[5,1,2]
    phi_q = param[:,2]
    phi_q = np.abs(phi_q.reshape(phi_q.shape[0],1,1)) # phi_shape: [5,1,1] # 这里我们需要取 phi的绝对值
    y_q = y_q.reshape(y_q.shape[0],y_q.shape[1],1)
    n_q = n_q.reshape(n_q.shape[0],n_q.shape[1],1)

    linpred_q = np.sum(np.multiply(x_q,b_q),axis=2) # linpred dim: [5, 3k]

    # linpred_q = linpred_q.reshape(linpred_q.shape[0],linpred_q.shape[1],1)

    end_time1 = time.time()
    # print("step 1 time : {} ms".format(1000*(end_time1 - start_time1)))

    mu_q = np.reciprocal(1 + np.exp(- ((linpred_q + np.matmul(randint_q, id_data_q)).reshape(5,zmat_q.shape[1],1)) -
                                        np.divide(zmat_q, np.matmul(np.sqrt(randint_prec_q), id_data_q).reshape(5,zmat_q.shape[1],1))  ))
    end_time2 = time.time()
    #print("step 2 time : {} ms".format(1000*(end_time2 - start_time1)))

    mu_q = np.clip(mu_q,1e-6,1-1e-6)
    end_time3 = time.time()
    #print("step 3 time : {} ms".format(1000*(end_time3 - start_time1)))

    alpha_1_q = mu_q / phi_q
    alpha_2_q = (1 - mu_q) / phi_q
    end_time4 = time.time()
    #print("step 4 time : {} ms".format(1000*(end_time4 - start_time1)))

    q_func_result = np.multiply( np.sum(special.betaln(alpha_1_q + y_q,alpha_2_q + n_q - y_q) -special.betaln(alpha_1_q, alpha_2_q),axis=1), # [5,3]
                           ghq_weights_q) # ghq_weights_q dim: [1,3]

    # 最后把 五个基因的 q-fun 的结果加在一起
    q_func_result = np.sum(q_func_result)

    end_time5 = time.time()
    #print("step 5 time : {} ms".format(1000*(end_time5 - start_time1)))
    return - q_func_result

# Derivative of the Q-function
def q_function_deri(param, x_q, y_q, n_q, zmat_q, ghq_weights_q, randint_q, randint_prec_q, id_data_q):
    start_time1 = time.time()
    #数据升维
    param = param.reshape(5,-1) # 我们输入的参数必须为一维的，这里我们把参数升级为原本的二维
    b_q = param[:,:2]
    b_q = b_q.reshape(b_q.shape[0],1,b_q.shape[1]) # b shape[5,1,2]
    phi_q = param[:,2]
    phi_q = phi_q.reshape(phi_q.shape[0],1,1) # phi_shape: [5,1,1]
    y_q = y_q.reshape(y_q.shape[0],y_q.shape[1],1)
    n_q = n_q.reshape(n_q.shape[0],n_q.shape[1],1)
    ghq_weights_q = ghq_weights_q.reshape(1, ghq_weights_q.shape[0], ghq_weights_q.shape[1]) # 升维成 dim: [1，1，3]

    linpred_q = np.sum(np.multiply(x_q,b_q),axis=2) # linpred dim: [5, 3k]

    mu_q = np.reciprocal(1 + np.exp(- ((linpred_q + np.matmul(randint_q, id_data_q)).reshape(5,zmat_q.shape[1],1)) -
                                        np.divide(zmat_q, np.matmul(np.sqrt(randint_prec_q), id_data_q).reshape(5,zmat_q.shape[1],1))  ))

    mu_q = np.clip(mu_q,1e-6,1-1e-6)

    alpha_1_q = mu_q / phi_q # dimension: [5, 3k, 1]
    alpha_2_q = (1 - mu_q) / phi_q # dimension: [5, 3k, 1]

    # b 的导数
    temp_b_q = special.digamma(alpha_1_q+y_q) - special.digamma(alpha_2_q+n_q-y_q) - special.digamma(alpha_1_q) + special.digamma(alpha_2_q) # dimension: (5, 30467, 3)
    # np.dot(x_q.transpose(),np.multiply(np.multiply(temp_b_q, mu_q),1-mu_q))/phi_q 这一步不太好处理，我们必须升维
    deri_b_q1 = x_q.reshape(x_q.shape[0],x_q.shape[1],x_q.shape[2],1) # 升维成 [5, 3k, 2, 1]
    deri_b_q2 = (np.multiply(np.multiply(temp_b_q, mu_q),1-mu_q)).reshape(temp_b_q.shape[0],temp_b_q.shape[1],1,temp_b_q.shape[2]) # dimension [5, 3k, 1, 3]
    deri_b_q3 = np.sum(np.multiply(deri_b_q1, deri_b_q2), axis = 1) /phi_q # dimension [5, 2, 3]
    deri_b_q4 = np.sum( np.multiply(deri_b_q3, ghq_weights_q), axis = 2) # dimension [5,2]

    # phi 的导数
    temp_phi_q = np.sum((special.digamma(alpha_1_q)-special.digamma(alpha_1_q+y_q))*mu_q +
              (special.digamma(alpha_2_q)-special.digamma(alpha_2_q+n_q-y_q))*(1 - mu_q)+
               special.digamma( 1/phi_q+ n_q )- special.digamma(1/phi_q),axis=1) / np.square(phi_q.reshape(-1,1))
    deri_phi_q = np.dot(temp_phi_q,ghq_weights_q.reshape(-1,1))

    # 合并导数，降维
    deri = np.concatenate((- deri_b_q4, - deri_phi_q),axis=1).reshape(-1)
    end_time = time.time()
    return deri

def vem(b_phi, sigm2, x, randint, id_data, y, n, ghq_weights, ghq_nodes,n_iter,n_lap):
  # E-M Algorithm
  for inter in range(n_iter):
    print("Interation: " + str(inter))
    start_time = time.time()

    # E-step
    start_e = time.time()
    b = b_phi[:,:2]
    b = b.reshape(b.shape[0],1,b.shape[1]) # shape[5,1,2]
    phi = b_phi[:,2]
    phi = phi.reshape(-1,1)
    for i in range(2):
      df = compute_randint_deriv_numpy(b, x, randint, id_data, y, n, phi)
      # print("df: " + str(df.shape))
      # print("randint: " + str(randint.shape))
      # print("sigm2: " + str(sigm2.shape))
      randint = randint - 0.9 * (df[:5,:] - randint / sigm2) / (df[5:,:] - 1 / sigm2)
    df = compute_randint_deriv_numpy(b, x, randint, id_data, y,n, phi)
    randint_prec = np.abs(df[5:,:] - 1 / sigm2)
    end_e = time.time()
    print("E-step : {} ms".format(1000*(end_e - start_e)) )

    # M - step
    start_m = time.time()
    zmat = np.multiply(np.zeros((y.shape[0],y.shape[1],3)) + 1, ghq_nodes).astype(np.float32)
    sigm2 =  (np.mean( randint * randint + np.reciprocal(randint_prec), axis = 1)).reshape(-1,1)
    # b_phi = np.array(b_phi).astype(np.float32)

    # b_phi = cp.asnumpy(cu_b_phi).astype(np.float32)
    cu_optim_result = minimize(q_function, b_phi.reshape(-1), method='L-BFGS-B',jac = q_function_deri,
                       args = (x, y, n, zmat, ghq_weights, randint, randint_prec, id_data))
    end_m= time.time()
    print("M-step : {} ms".format(1000*(end_m - start_m)))
    b_phi = (cu_optim_result.x).reshape(5,-1)
    func =  cu_optim_result.fun
    end_time = time.time()
    print("  b: " + str(b_phi[:,:2]))
    print("  phi: " + str(b_phi[:,2]))
    print("  sigma2: " + str(sigm2) )
    print("  Likelihood: " + str(func) )
    print("  Time :{} ms".format(1000*(end_time - start_time)))


In [ ]:
vem(param, sigm2, x, randint, id_data, y, n, ghq_weights, ghq_nodes,n_iter= 10 ,n_lap=2)

Interation: 0
E-step : 1038.754940032959 ms
M-step : 13872.95913696289 ms
  b: [[ 0.19519112 -0.01905776]
 [ 0.11463712  0.17952667]
 [ 0.10415416  0.17119443]
 [ 0.17931554  0.00526237]
 [ 0.18682     0.00238899]]
  phi: [0.38046442 1.53292178 2.72477073 0.45338728 0.60883303]
  sigma2: [[0.09711352]
 [0.15640325]
 [0.11740644]
 [0.14650798]
 [0.13069939]]
  Likelihood: 275296.7171325335
  Time :14913.083791732788 ms
Interation: 1
E-step : 1004.9796104431152 ms
M-step : 7830.073118209839 ms
  b: [[ 0.19886452 -0.053214  ]
 [ 0.13587605  0.15886614]
 [ 0.09379463  0.13973251]
 [ 0.17575934 -0.01753316]
 [ 0.19449412 -0.0311578 ]]
  phi: [0.37943418 1.5393189  2.7346428  0.45240433 0.60828273]
  sigma2: [[0.07458628]
 [0.14982347]
 [0.0938144 ]
 [0.14963128]
 [0.12562465]]
  Likelihood: 275283.1115171886
  Time :8836.962461471558 ms
Interation: 2
E-step : 1020.0784206390381 ms
M-step : 4498.926639556885 ms
  b: [[ 0.18680989 -0.05525287]
 [ 0.14812224  0.1574582 ]
 [ 0.07355253  0.13122

# Gene 5 + Cupy
## The True Game Begins !!!

In [71]:
def cu_trigamma(cu_array): # Test numpy(): trigamma(x_cpu) + x_gpu
  n = 1
  polygamma_cui = (-1.0)**(n+1) * _gamma.gamma(n+1.0) * _zeta.zeta(n+1.0, cu_array)
  return polygamma_cui.astype(cp.float32)

def cu_compute_randint_deriv_numpy(cu_e_b, cu_e_x, cu_e_randint, cu_id_data, cu_e_y, cu_e_n, cu_e_phi):
  cu_e_mu = cp.reciprocal( 1 + cp.exp(- cp.sum(cp.multiply(cu_e_x, cu_e_b), axis=2) - cp.matmul(cu_e_randint, cu_id_data)))

  cu_e_mu = cp.clip(cu_e_mu,0.0000001, 0.9999999)

  cu_temp_1 = (special.digamma(cu_e_mu / cu_e_phi + cu_e_y) -
               special.digamma((1 - cu_e_mu)/ cu_e_phi + cu_e_n - cu_e_y) -
               special.digamma(cu_e_mu / cu_e_phi) +
               special.digamma((1 - cu_e_mu) / cu_e_phi) )
  cu_temp_2 = ( cu_trigamma(cu_e_mu / cu_e_phi + cu_e_y ) +
             cu_trigamma(( 1 - cu_e_mu ) / cu_e_phi + cu_e_n - cu_e_y) -
             cu_trigamma(cu_e_mu / cu_e_phi) -
             cu_trigamma(( 1 - cu_e_mu)/ cu_e_phi) )

  cu_v1 = cu_temp_1 * cu_e_mu * (1 - cu_e_mu) / cu_e_phi
  cu_v2 = (cp.multiply(cp.multiply(cu_temp_2, cp.multiply(cu_e_mu,cu_e_mu)), cp.multiply(1- cu_e_mu, 1- cu_e_mu)) / (cu_e_phi * cu_e_phi) +
           cp.multiply(cp.multiply(cp.multiply(cu_temp_1, 1 - 2 * cu_e_mu), cu_e_mu), 1 - cu_e_mu) / cu_e_phi )
  cu_v1 = cp.matmul(cu_v1, cu_id_data.T)
  cu_v2 = cp.matmul(cu_v2, cu_id_data.T)
  cu_v =  cp.concatenate((cu_v1,cu_v2),axis=0)
  return cu_v


# Q-function for numpy
def cu_q_function(param, cu_x_q, cu_y_q, cu_n_q, cu_zmat_q, cu_ghq_weights_q, cu_randint_q, cu_randint_prec_q, cu_id_data_q, cu_num_genes_q):
    start_time1 = time.time()
    #print("qfunction param : \n"+ str(param.reshape(-1,3)))
    #数据升维
    cu_param = ((cp.asarray(param)).astype(cp.float32)).reshape(cu_num_genes_q,-1) # 我们输入的参数必须为一维的，这里我们把参数升级为原本的二维
    cu_b_q = cu_param[:,:2]
    cu_b_q = cu_b_q.reshape(cu_b_q.shape[0],1,cu_b_q.shape[1]) # cu_b shape[5,1,2]
    cu_phi_q = cu_param[:,2]
    cu_phi_q = cp.abs(cu_phi_q.reshape(cu_phi_q.shape[0],1,1)) # cu_phi_shape: [5,1,1] # 这里我们需要取 phi的绝对值
    cu_y_q = cu_y_q.reshape(cu_y_q.shape[0],cu_y_q.shape[1],1)
    cu_n_q = cu_n_q.reshape(cu_n_q.shape[0],cu_n_q.shape[1],1)

    cu_linpred_q = cp.sum(cp.multiply(cu_x_q, cu_b_q),axis=2) # cu_linpred dim: [5, 3k]

    # linpred_q = linpred_q.reshape(linpred_q.shape[0],linpred_q.shape[1],1)

    end_time1 = time.time()
    # print("step 1 time : {} ms".format(1000*(end_time1 - start_time1)))

    cu_mu_q = cp.reciprocal(1 + cp.exp(- ((cu_linpred_q + cp.matmul(cu_randint_q, cu_id_data_q)).reshape(cu_num_genes_q,cu_zmat_q.shape[1],1)) -
                                        cp.divide(cu_zmat_q, cp.matmul(cp.sqrt(cu_randint_prec_q), cu_id_data_q).reshape(cu_num_genes_q, cu_zmat_q.shape[1],1))  ))
    end_time2 = time.time()
    #print("step 2 time : {} ms".format(1000*(end_time2 - start_time1)))

    cu_mu_q = cp.clip(cu_mu_q,1e-6,1-1e-6)
    end_time3 = time.time()
    #print("step 3 time : {} ms".format(1000*(end_time3 - start_time1)))

    cu_alpha_1_q = cu_mu_q / cu_phi_q
    cu_alpha_2_q = (1 - cu_mu_q) / cu_phi_q
    end_time4 = time.time()
    #print("step 4 time : {} ms".format(1000*(end_time4 - start_time1)))

    cu_q_func_result = cp.multiply( cp.sum(special.betaln(cu_alpha_1_q + cu_y_q, cu_alpha_2_q + cu_n_q - cu_y_q) -
                                           special.betaln(cu_alpha_1_q, cu_alpha_2_q), axis=1), # [5,3]
                           cu_ghq_weights_q) # ghq_weights_q dim: [1,3]

    # 最后把 五个基因的 q-fun 的结果加在一起
    #print("q-function: \n" + str(- cp.sum(cu_q_func_result)))
    cu_q_func_result = cp.sum(cu_q_func_result)

    end_time5 = time.time()
    #print("step 5 time : {} ms".format(1000*(end_time5 - start_time1)))
    #print("q-function: \n" + str(- cu_q_func_result))
    return - (cp.asnumpy(cu_q_func_result)).astype(np.float32)


# Derivative of the Q-function
def cu_q_function_deri(param, cu_x_q, cu_y_q, cu_n_q, cu_zmat_q, cu_ghq_weights_q, cu_randint_q, cu_randint_prec_q, cu_id_data_q, cu_num_genes_q):
    start_time1 = time.time()
    #数据升维
    #print("q derivative param: \n" + str(param.reshape(-1,3)))
    cu_param = ((cp.asarray(param)).astype(cp.float32)).reshape(cu_num_genes_q,-1) # 我们输入的参数必须为一维的，这里我们把参数升级为原本的二维
    cu_b_q = cu_param[:,:2]
    cu_b_q = cu_b_q.reshape(cu_b_q.shape[0],1,cu_b_q.shape[1]) # cu_b shape[5,1,2]
    cu_phi_q = cu_param[:,2]
    cu_phi_q = cu_phi_q.reshape(cu_phi_q.shape[0],1,1) # cu_phi_shape: [5,1,1]
    cu_y_q = cu_y_q.reshape(cu_y_q.shape[0], cu_y_q.shape[1], 1)
    cu_n_q = cu_n_q.reshape(cu_n_q.shape[0], cu_n_q.shape[1],1)
    cu_ghq_weights_q = cu_ghq_weights_q.reshape(1, cu_ghq_weights_q.shape[0], cu_ghq_weights_q.shape[1]) # 升维成 dim: [1，1，3]

    cu_linpred_q = cp.sum(cp.multiply(cu_x_q, cu_b_q),axis=2) # linpred dim: [5, 3k]

    cu_mu_q = cp.reciprocal(1 + cp.exp(- ((cu_linpred_q + cp.matmul(cu_randint_q, cu_id_data_q)).reshape(cu_num_genes_q, cu_zmat_q.shape[1],1)) -
                                           cp.divide(cu_zmat_q, cp.matmul(cp.sqrt(cu_randint_prec_q), cu_id_data_q).reshape(cu_num_genes_q, cu_zmat_q.shape[1],1))  ))

    cu_mu_q = cp.clip(cu_mu_q,1e-6,1-1e-6)

    cu_alpha_1_q = cu_mu_q / cu_phi_q # dimension: [5, 3k, 1]
    cu_alpha_2_q = (1 - cu_mu_q) / cu_phi_q # dimension: [5, 3k, 1]

    # b 的导数
    cu_temp_b_q = ( special.digamma(cu_alpha_1_q+ cu_y_q) - special.digamma(cu_alpha_2_q+ cu_n_q - cu_y_q)
                    - special.digamma(cu_alpha_1_q) + special.digamma(cu_alpha_2_q) ) # dimension: (5, 30467, 3)
    # np.dot(x_q.transpose(),np.multiply(np.multiply(temp_b_q, mu_q),1-mu_q))/phi_q 这一步不太好处理，我们必须升维
    cu_deri_b_q1 = cu_x_q.reshape(cu_x_q.shape[0], cu_x_q.shape[1], cu_x_q.shape[2],1) # 升维成 [5, 3k, 2, 1]
    cu_deri_b_q2 = (cp.multiply(cp.multiply(cu_temp_b_q, cu_mu_q),1 - cu_mu_q)).reshape(cu_temp_b_q.shape[0], cu_temp_b_q.shape[1], 1, cu_temp_b_q.shape[2]) # dimension [5, 3k, 1, 3]
    cu_deri_b_q3 = cp.sum(cp.multiply(cu_deri_b_q1, cu_deri_b_q2), axis = 1) /cu_phi_q # dimension [5, 2, 3]
    cu_deri_b_q4 = cp.sum(cp.multiply(cu_deri_b_q3, cu_ghq_weights_q), axis = 2) # dimension [5,2]

    # phi 的导数
    cu_temp_phi_q = cp.sum((special.digamma(cu_alpha_1_q)-special.digamma(cu_alpha_1_q+cu_y_q))*cu_mu_q +
                 (special.digamma(cu_alpha_2_q)-special.digamma(cu_alpha_2_q+ cu_n_q- cu_y_q))*(1 - cu_mu_q)+
                 special.digamma( 1/cu_phi_q+ cu_n_q )- special.digamma(1/cu_phi_q),axis=1) / np.square(cu_phi_q.reshape(-1,1))
    cu_deri_phi_q = cp.dot(cu_temp_phi_q, cu_ghq_weights_q.reshape(-1,1))
    #print("q deri phi: \n " + str(cu_deri_phi_q))
    # 这里我们把q function写成了关于phi的偶函数，所以q-函数关于phi的导数，我门需要根据原来的变化变一下, 否则q-function的值会爆炸
    #cu_phi_sign = (cp.sign(cu_param[:,2])).reshape(-1,1)
    #cu_deri_phi_q = cu_deri_phi_q * cu_phi_sign
    #print("q deri phi after sign: \n " + str(cu_deri_phi_q))
    #print()
    #print()

    # 合并导数，降维
    cu_deri = cp.concatenate((cu_deri_b_q4, cu_deri_phi_q),axis=1).reshape(-1)
    end_time = time.time()

    #print("derivative: \n" + str(- cu_deri.reshape(-1,3)))
    #print()
    return - (cp.asnumpy(cu_deri)).astype(np.float32)



def cu_vem(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_ghq_weights, cu_ghq_nodes, cu_n_iter, cu_n_lap, cu_num_genes):
  # E-M Algorithm
  for inter in range(cu_n_iter):
    print("Interation: " + str(inter))
    start_time = time.time()

    # E-step
    start_e = time.time()
    cu_b = cu_b_phi[:,:2]
    cu_b = cu_b.reshape(cu_b.shape[0], 1, cu_b.shape[1]) # shape[num_genes,1,2]
    cu_phi = cu_b_phi[:,2]
    cu_phi = cu_phi.reshape(-1,1)
    for i in range(2):
      cu_df = cu_compute_randint_deriv_numpy(cu_b, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
      cu_randint = cu_randint - 0.9 * (cu_df[:cu_num_genes,:] - cu_randint / cu_sigm2) / (cu_df[cu_num_genes:,:] - 1 / cu_sigm2)
    cu_df = cu_compute_randint_deriv_numpy(cu_b, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
    cu_randint_prec = cp.abs(cu_df[cu_num_genes:,:] - 1 / cu_sigm2)
    end_e = time.time()
    print(" E-step : {} ms".format(1000*(end_e - start_e)))

    #print("cu_df: " + str(cu_df.shape))
    #print(cu_df[:50,:])

    # M - step
    start_m = time.time()
    cu_zmat = cp.multiply(cp.zeros((cu_y.shape[0],cu_y.shape[1],3)) + 1, cu_ghq_nodes).astype(cp.float32)
    cu_sigm2 =  (cp.mean( cu_randint * cu_randint + cp.reciprocal(cu_randint_prec), axis = 1)).reshape(-1,1)

    cu_b_phi = cp.asnumpy(cu_b_phi).astype(np.float32)
    low_bound = (np.ones_like(cu_b_phi) * np.array([-np.inf, -np.inf, 1e-6]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是下界
    up_bound = (np.ones_like(cu_b_phi) * np.array([np.inf, np.inf, np.inf]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是上界

    cu_optim_result = minimize(cu_q_function, cu_b_phi.reshape(-1), method='L-BFGS-B', bounds = Bounds(low_bound,up_bound),
                               jac = cu_q_function_deri,
                               options={'ftol':1e-8, 'maxls':100},
                               args = (cu_x, cu_y, cu_n, cu_zmat, cu_ghq_weights, cu_randint, cu_randint_prec, cu_id_data, cu_num_genes))

    end_m= time.time()
    print(" M-step : {} ms".format(1000*(end_m - start_m)))
    cu_b_phi = cp.asarray((cu_optim_result.x).reshape(cu_num_genes,-1))
    cu_func =  cu_optim_result.fun
    end_time = time.time()
    #print("  b: " + str(cu_b_phi[:,:2]))
    #print("  phi: " + str(cu_b_phi[:,2]))
    # print("  sigma2: " + str(cu_sigm2) )
    # print("  Likelihood: " + str(cu_func) )
    print("  Time :{} ms".format(1000*(end_time - start_time)))
    print(cu_optim_result)
    #print(cu_optim_result.x)
    print()
  return cu_b_phi, cu_sigm2

In [54]:
# Import Data
# get x, y, n
ynxid_data = np.load("ynxid_allgenes.npy")
ynxid_data = ynxid_data[:,1:] # allgenes data 的时候用一下，数据没清理干净

# get id
id_data = np.load("id_alldata.npy")
id_data = id_data[1:,:] # 当处理all genes时，ID把第一个去掉，数据没清理干净

id_data = id_data.T
cu_id_data = cp.asarray(id_data)

In [77]:
my_list = [i for i in range(101)]
print(my_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [82]:
gene_index = [i for i in range(1001)]

total_genes = int((ynxid_data.shape[0] - 2) / 2)
#cu_ynxid_data = cp.asarray(ynxid_data[:2000,:]) # 这里为了解决内存，我门先不转化
cu_y = cp.asarray(ynxid_data[gene_index,:])
cu_n = cp.asarray(ynxid_data[ np.array(gene_index) + total_genes,:])

cu_x = cp.zeros((len(gene_index),cu_y.shape[1],2)).astype(cp.float32)
for i in range(len(gene_index)):
  cu_x_0 = cp.zeros((cu_y.shape[1])) + 1
  cu_x_1 = cp.asarray(ynxid_data[-2,:])
  cu_x[i,:,:] = cp.hstack((cu_x_0.reshape(-1, 1), cu_x_1.reshape(-1, 1))).astype(cp.float32)

# Now, let's start our Game !

# Preparing Data
cu_param = cp.concatenate(((cp.zeros(len(gene_index))+ 0.08).reshape(-1,1), # b0 = 0.03
                           (cp.zeros(len(gene_index))+ 0.5).reshape(-1,1),  # b1 = 0.4
                           (cp.zeros(len(gene_index))+ 0.35).reshape(-1,1)), axis=1) # phi = 0.72
cu_sigm2 = cp.zeros((len(gene_index),1)) + 0.5

cu_randint = cp.zeros((len(gene_index),cu_id_data.shape[0]), dtype=cp.float32)

cu_ghq_weights = cp.array([[0.1666667, 0.6666667, 0.1666667]]).astype(cp.float32).reshape(1,3)

cu_ghq_nodes = cp.array([[-1.732051, -2.045201e-16, 1.732051]]).astype(cp.float32).reshape(1,3)


In [83]:
my_b_phi, my_sigm2 =  cu_vem(cu_param, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_ghq_weights, cu_ghq_nodes,
                             cu_n_iter= 60 ,cu_n_lap=2, cu_num_genes= len(gene_index))


Interation: 0
 E-step : 17.431974411010742 ms


KeyboardInterrupt: 

In [ ]:
my_b_phi

In [ ]:
my_sigm2